In [3]:
import pandas as pd
twtdata = pd.read_csv('reduced_dataset.csv', engine='pyarrow')
# twtdata.head(20)
twtdata = twtdata.dropna()

twtdata = twtdata[twtdata['text'] != '']
twtdata = twtdata.iloc[1000:]


In [12]:
key = ENTER_API_KEY_HERE
instruction_text = ''''The following is a block of text that is from a twitter poll. You have to classify it in one of the following categories: {'other', 'Social', 'political', 'economics', 'entertainment'}. Your reply must only contain one of these 5 words, ONE word only'''

In [21]:
len(twtdata)
twtdata_sample = twtdata.sample(n=1000)
twtdata_sample['combined_text'] = instruction_text + twtdata_sample['text'] + ' ' + twtdata_sample['polls'].astype(str)
import openai
openai.api_key = key

def get_gpt4_response(text):
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=text,
      max_tokens=100
    )
    return response.choices[0].text.strip()

twtdata_sample['gpt4_response'] = twtdata_sample['combined_text'].apply(get_gpt4_response)

print(twtdata_sample[['gpt3_response', 'Human_Eval']])


      gpt3_response     Human_Eval
2588         Social          other
2668      Economics          other
3341  Entertainment  entertainment
1353         Social          other
3875          other         Social
...             ...            ...
1296  Entertainment  entertainment
1377         Social          other
2277  Entertainment  entertainment
3542  Entertainment         Social
2132  Entertainment  entertainment

[1000 rows x 2 columns]


In [2]:
twtdata_sample.to_csv('final_dataframe.csv', index=False)

NameError: name 'twtdata_sample' is not defined

In [5]:
twtdata_sample = pd.read_csv('final_dataframe.csv')

# Encoding the 'Human_Eval' and 'gpt3_response' columns using one-hot encoding
human_eval_encoded = pd.get_dummies(twtdata_sample['Human_Eval'], prefix='Human_Eval')
gpt3_response_encoded = pd.get_dummies(twtdata_sample['gpt3_response'], prefix='gpt3_response')

# Concatenating the original dataframe with the encoded dataframes
twtdata_sample_encoded = pd.concat([twtdata_sample, human_eval_encoded, gpt3_response_encoded], axis=1)

from sklearn.metrics import accuracy_score
# Converting the 'Human_Eval' and 'gpt3_response' columns to string type
twtdata_sample['Human_Eval'] = twtdata_sample['Human_Eval'].astype(str)
twtdata_sample['gpt3_response'] = twtdata_sample['gpt3_response'].astype(str)
# Calculating the accuracy score between 'Human_Eval' and 'gpt3_response'
accuracy = accuracy_score(twtdata_sample['Human_Eval'], twtdata_sample['gpt3_response'])

# Printing the accuracy score
print("Accuracy: ", accuracy)


Accuracy:  0.46
